In [ ]:
#USING MOBILENETV2 WITH FINETUNNING(UNFREEZE)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# Load base MobileNetV2 model
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model initially

# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Helps prevent overfitting
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train only the top layer
history = model.fit(train_data, validation_data=val_data, epochs=10)

# 🔧 Fine-tuning: Unfreeze some top layers of the base model
base_model.trainable = True

# Optional: Freeze lower layers if needed (to retain low-level features)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Compile again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
fine_tune_history = model.fit(train_data, validation_data=val_data, epochs=10)